In [5]:
import psycopg2
import uuid
from datetime import datetime, timedelta

# Подключение к базе данных PostgreSQL
def get_connection():
    return psycopg2.connect(
        dbname="app_db",
        user="user",
        password="8vjsiecj(#D2",
        host="db",
        port="5432"
    )

# Получение данных для 2025-01-02
def fetch_data_for_date(date):
    connection = get_connection()
    cursor = connection.cursor()
    
    query = """
        SELECT partner_id, qual_type, good_id
        FROM betapro_data
        WHERE date = %s
    """
    
    cursor.execute(query, (date,))
    data = cursor.fetchall()
    
    cursor.close()
    connection.close()
    
    return data

# Вставка новых записей
def insert_missing_data(date, data):
    connection = get_connection()
    cursor = connection.cursor()
    
    # Генерация новых записей для 2025-01-03 и 2025-01-04
    for partner_id, qual_type, good_id in data:
        for new_date in [date + timedelta(days=1), date + timedelta(days=2)]:  # 2025-01-03, 2025-01-04
            new_uuid = str(uuid.uuid4())
            insert_query = """
                INSERT INTO betapro_data (uuid, date, partner_id, qual_type, qnt, qnt2, qnt3, qnt4, qnt5, qnt6, qnt7, qnt8, "Свободный остаток", good_id)
                VALUES (%s, %s, %s, %s, 0, 0, 0, 0, 0, 0, 0, 0, 0, %s)
            """
            cursor.execute(insert_query, (new_uuid, new_date, partner_id, qual_type, good_id))
    
    connection.commit()
    cursor.close()
    connection.close()


In [6]:
# Дата для поиска пропущенных данных (2025-01-02)
base_date = datetime(2025, 1, 2)

# Получаем данные для 2025-01-02
data_for_2025_01_02 = fetch_data_for_date(base_date)

# Вставляем данные для 2025-01-03 и 2025-01-04
insert_missing_data(base_date, data_for_2025_01_02)